# **Lecture 7: Generative Algorithms**
# **Applied Machine Learning**

## **Part 1: Generative Models**
In this lecture, we are going to look at generative algorithms and their application to classification.
We will start by defining the concept of a *generative model*

### **Review: Components of A Supervised Machine Learning Problem**

At a high level, a supervised machine learning problem has the following structure:
$$\underbrace{\text{Training Dataset}}_{\text{Attributes + Features}} + \underbrace{\text{Learning Algorithm}}_{\text{Model class + Objective + Optimizer}} \to \text{Predictive Model}$$



### **Review: Probabilistic Models**

A (parametric) probabilistic model with parameters $\theta$ is a probability distribution:

$$P_{\theta}(x,y) : \mathcal{X} \times \mathcal{Y} \to [0,1]$$

This model can approximate the data distribution $\mathbb{P}(x,y)$.

If we know $P_{\theta}(x,y)$, we can compute predictions using the formula:

$$P_{\theta}(y \mid x) = \frac{P_{\theta}(x,y)}{P_{\theta}(x)} = \frac{P_{\theta}(x,y)}{\sum_{y \in \mathcal{Y}}P_{\theta}(x,y)}.$$

### **Review: Maximum Likelihood Learning**
In order to fit probabilistic models, we use the following objective:
$$\max_{\theta}\mathbb{E}_{x \sim \mathbb{P}}\log P_{\theta}(x,y).$$

This seeks to find a model that assigns high probability to the training data.

### **Review: Conditional Probabilistic Models**
Alternatively, we may define a model of the conditional probabilistic distribution:
$$P_{\theta}(y \mid x): \mathcal{X} \times \mathcal{Y} \to [0,1].$$

These are trained using conditional maximum likelihood:
$$\max_{\theta}\mathbb{E}_{x \sim \mathbb{P}} \log P_{\theta}(y \mid x).$$

This seeks to find a model that assigns high probability to the target $y$ for each $x$.

Logistic Regression is an example of this approach.

## **Discriminative vs. Generative Models**
These two types of models are also known as **generative** and **discriminative**.


$$\begin{align*}
\underbrace{P_{\theta}(x,y) : \mathcal{X} \times \mathcal{Y} \to [0,1]}_{\text{generative model}} & \; \; &
\underbrace{P_{\theta}(y \mid x): \mathcal{X} \times \mathcal{Y} \to [0,1]}_{\text{discriminative model}}
\end{align*}$$


*   The models parametrize different kinds of probabilities.
*   They involve different training objectives and make different predictions.
*   Their uses are different (e.g., prediction, generation, etc.); more later!



### **Classification Dataset: Iris Flowers**

To demonstrate the two approaches, we are going to use the Iris flower.



In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
import warnings
warnings.filterwarnings('ignore')

# Load the iris dataset
iris = datasets.load_iris(as_frame=True)

# Print part of dataset
iris_X, iris_y = iris.data, iris.target
pd.concat([iris_X, iris_y], axis=1).head()

If we only consider the first two feature columns, we can visualize the dataset in 2D

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12,4]

# Create 2d version of dataset
X = iris_X.to_numpy()[:,:2]
x_min, x_max = X[:,0].min() - .5, X[:,0].max() + .5
y_min, y_max = X[:,1].min() - .5, X[:,1].max() + .5

# Plot also the training points
p1 = plt.scatter(X[:,0], X[:,1], c=iris_y, edgecolor='k', s=40, cmap=plt.cm.Paired)
plt.xlabel('Sepal lenght')
plt.ylabel('Sepal width')
plt.legend(handles=p1.legend_elements()[0], labels=['Setosa', 'Versicolor', 'Virginica'], loc='lower right')

### **Example: Disciminative Model**
An example of discriminative model are logistic regression or softmax regression

*   Discriminative models directly partition the feature space into regions associated with each class ans separated by the decision boundaries.
*   Given a feature $x$, disciminative models directly map to the predicted classes (e.g., via the function $\sigma(\theta^\top x)$ for logistic regression)



In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5, multi_class='multinomial')

# Create an instance of softmax regression and fit the data
logreg.fit(X, iris_y)
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into the color plot
Z = Z.reshape(xx.shape)
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X[:,0], X[:,1], c=iris_y, edgecolor='k', s=60, cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

### **Example: Generative Models**

Generative modeling can be seen as taking a different approach:


1.   In the Iris example, we first build a model of how each type of flower looks, i.e., we can learn the distribution.
$$p(x \mid y = k) \; \text{for each class} \; k.$$

It defines a model of how each flower is **generated**, hence the name
2.   Given a new flower datapoint $x'$, we can match each against each flower model and find the type of flower that looks most similar to it. Mathematically, this corresponds to:
$$\begin{align*}
\arg\max_{y}\log p(y \mid x) &= \arg \max_{y}\log \frac{ p(x \mid y)p(y)}{p(x)}\\
&= \arg \max_y \log p(x \mid y)p(y),
\end{align*}$$

where we have applied Bayes' rule in the first line.

### **Generative vs. Discriminative Approach**

How do we know which approach is better?

*   If we only care about prediction, we don't need to know the model of $P(x)$. We can solve precisely the problem we care about.
 * Discriminative models will be often more accurate. 
*   If we care about other tasks (e.g., generation, dealing with missing values, etc.) or if we know the true model is generative, we want to use the generative approach.



## **Part 2: Gaussian Discriminant Analysis**

We are now going to conitue our discussion of classification


*   We wiil see a new classification algorithm, Gaussian Discriminat Analysis.
*   This will be our first example of generative machine learning model.



### **Review: Classification**
Considering a training dataset $\mathcal{D} = \{(x^{(i)}, y^{(i)}), i = 1,2,\dots, n\}$.

We distinguishing two types of supervised learning problems depending on the targets $y^{(i)}$.


1.   **Regression**: The target variable $y \in \mathcal{Y}$ is continuous: $\mathcal{Y} \in \mathbb{R}$ 
2.   **Classification**: The target variable $y$ is discrete and takes on one of $K$ possible values: $\mathcal{Y} = \{y_1, y_2, \dots, y_K\}$. Each discrete value corresponds to a class that we want to predict.



### **Review: Generative Models**
There are two types of probabilistic models: *generative* and *disciminative*.

$$\begin{align*}
\underbrace{P_{\theta}(x,y): \mathcal{X} \times \mathcal{Y} \to [0,1]}_{\text{generative model}} & \; \; & \underbrace{P_{\theta}(y \mid x): \mathcal{X} \times \mathcal{Y} \to [0,1]}_{\text{discriminative model}}
\end{align*}$$


*   They involve different training objectives and make different predictions.
*   Their uses are different (e.g. generation, prediction, etc.)



### **Mixtures of Gaussians**

A mixture of $K$ Gaussians is a distribution $P(X)$ of the form:
$$\phi_1\mathcal{N}(x; \mu_1, \Sigma_1) + \phi_2\mathcal{N}(x; \mu_2, \Sigma_2) + \dots + \phi_K\mathcal{N}(x; \mu_K, \Sigma_K).$$

*   Each $\mathcal{N}(x; \mu_K, \Sigma_K)$ is a (multivariate) Gaussisan distribution with mean $\mu_K$ and covariance $\Sigma_K$.
*   The $\phi_K$ are weights, and the above sum is a weighted average of the $K$ Gaussians.



We can easily visualize this in 1D

In [ ]:
def N(x, mu, sigma):
  return np.exp(-.5*(x -mu)**2/sigma**2)/np.sqrt(2*np.pi*sigma)

def mixture(x):
  return 0.6*N(x,mu=1,sigma=0.5) + 0.4*N(x,mu=-1, sigma=0.5)

xs, xs1, xs2 = np.linspace(-3,3), np.linspace(-1,3), np.linspace(-3,1)

plt.subplot(121)
plt.plot(xs1, 0.6*N(xs1,mu=1,sigma=0.5), label='First Gaussian', alpha=0.7)
plt.plot(xs2, 0.4*N(xs2,mu=-1,sigma=0.5), label='Second Gaussian', alpha=0.7)
plt.legend()

plt.subplot(122)
plt.plot(xs, mixture(xs), label='Mixture Gaussians', linewidth=2)
plt.legend()


### **Gaussian Disciminant Model**

We may use this approach to define a model $P_{\theta}$. This will be a basis of an algorithm called Gaussian Discriminant analysis.


*   The distribution over classes is [Categorical](), denoted $\text{Categorical}(\phi_1, \phi_2, \dots, \phi_K)$. Thus, $P_{\theta}(y=k)=\phi_k.$
*   The conditional probability $P_{\theta}(x \mid y=k)$ of the data under the class $k$ is a [multivariate Gaussian]() $\mathcal{N}(x; \mu_k, \Sigma_k)$ with mean and covariance $\mu_k, \Sigma_k$.

Thus, $P_{\theta}(x,y)$ is a mixture of $K$ Guassians:
$$P_{\theta}(x,y) = \sum_{k=1}^K P_{\theta}(y=k)P_{\theta}(x \mid y=k) = \sum_{k=1}^K \phi_K\mathcal{N}(x; \mu_K, \Sigma_K)$$

Intuitively, this model defines a story for how the data was generated. To obtain a data point.

*   First, we sample a class $y \sim \text{Categorical}(\phi_1, \phi_2, \dots, \phi_K)$ with class proportion given by the $\phi_k$
*   Then, we sample an $x$ from a Guassian distribution $\mathcal{N}(\mu_k, \Sigma_k)$ specific to that class.

Such a story can be constructed for most generative algorithms and helps understand them.







### **Classification Dataset: Iris Flowers**

To demonstrate this approach, we are going to use the Iris flower dataset.



In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import datasets

# Load the Iris dataset
iris = datasets.load_iris(as_frame=True)

# print part of the dataset
iris_X, iris_y = iris.data, iris.target
pd.concat([iris_X, iris_y], axis=1).head()

If we only care about the first two features we can visualize the dataset in 2D

In [ ]:
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 4]

# create 2d version of dataset
X = iris_X.to_numpy()[:,:2]
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

# Plot also the training points
p1 = plt.scatter(X[:, 0], X[:, 1], c=iris_y, edgecolor='k', s=60, cmap=plt.cm.Paired)
plt.xlabel('Sepal Length (cm)')
plt.ylabel('Sepal Width (cm)')
plt.legend(handles=p1.legend_elements()[0], labels=['Setosa', 'Versicolour', 'Virginica'], loc='lower right')

### **Example: Iris Flower Classidication**
Let's see how this approach can be used in practice on Iris dataset.

*   We will "guess" a good set of parameters for a Gaussian Discriminant Model.
*   We will sample from the model and compare to the true data.



In [ ]:
s = 100 # number of samples
K = 3 # number of classes
d = 2 # number of features

# Guess the parameters
phi = 1/K*np.ones(K,)
mus = np.array(
    [[5.0, 3.5],
     [6.0, 2.5],
     [6.5, 3.0]
     ])
Sigmas = 0.05*np.tile(np.reshape(np.eye(2), (1,2,2)), (K,1,1))

# Generate data from this model
ys = np.random.multinomial(n=1, pvals=phi, size=(s,)).argmax(axis=1)
xs = np.zeros([s,d])
for k in range(K):
  nk = (ys==k).sum()
  xs[ys==k,:] = np.random.multivariate_normal(mus[k], Sigmas[k], size=(nk,))

print(xs[:10])


In [ ]:
plt.subplot(121)
plt.title('Model Sample')
plt.scatter(xs[:,0], xs[:,1], c=ys, cmap=plt.cm.Paired)
plt.scatter(X[:,0], X[:,1], c=iris_y, edgecolor='k', s=40, cmap=plt.cm.Paired, alpha=0.15)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

# Plot also the training points
plt.subplot(122)
plt.title('Training Dataset')
plt.scatter(X[:,0], X[:,1], c=iris_y, edgecolor='k', s=40, cmap=plt.cm.Paired, alpha=1)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')


*   Our Gaussian Discriminant Model generate data that looks not unlike the real data
*   Let's now see how we can learn parameters from data and use the model to make predictions.



## **Part 3: Gaussian Discriminant Analysis: Learning**

We continue our discussion of Gaussian Disciminant analysis, and look at:

*   How to learn parameters of the mixture model
*   How to use model to make predictions



### **Review: Classification**
Consider a training dataset $\mathcal{D}$.
We distiguish between the two types of supervised learning problems depending on the targets $y^{(i)}$


1.   **Regression:** The target variable $y \in \mathcal{Y}$ is continuous: $\mathcal{Y} \in \mathbb{R}$
2.   **Classification:** The target variable $y$ is disrete and take on one of $K$ possible values $\mathcal{Y} = \{y_1, y_2, \dots, y_K\}$. Each discrete value corresponds to a *class* that we want to predict.



### **Review: Gaussian Discriminant Analysis**
We may define a model $P_{\theta}$ as follows. This will be the basis of an algorithm called Gaussian Discriminant Analysis.

*   The distribution over class is [Categorical](), denoted $\text{Categorical}(\phi_1, \phi_2, \dots, \phi_K)$. Thus, $P_{\theta}(y=k) = \phi_k$
*   The conditional probability $P(x \mid y = k)$ of the data under the class $k$ is a [multivariate Gaussian]() $\mathcal{N}(x;\mu_k,\Sigma_k)$ with mean and covariance $\mu_k, \Sigma_k$

Thus, $P_{\theta}(x,y)$ is a mixture of $K$ Gaussians
$$P_{\theta}(x,y) = \sum_{k=1}^KP_{\theta}(y=k)P(x \mid y = k) = \sum_{k=1}^K \phi_k\mathcal{N}(x;\mu_k,\Sigma_k).$$



### **Review: Maximum Likelihood Learning**

In order to fit probabilistic models, we use the following objective:
$$\max_{\theta}\mathbb{E}_{x,y \sim \mathbb{P}}\log P_{\theta}(x,y).$$

This seeks to find a model that assigns high probability to the training data.

Let's use maximum likelihood to fit Gaussian Discriminant Model. Note that model parameters $\theta$ are the union of the parameters of each sub-model:
$$\theta = \{\mu_1, \Sigma_1, \phi_1, \dots, \mu_K, \Sigma_K, \phi_K\}.$$

Mathematically, the components of the model $P_{\theta}(x,y)$ as follows:
$$\begin{align*}
P_{\theta}(y) &= \frac{\prod_{k=1}^K\phi_k^{\mathbb{I}\{y=y_k\}}}{\sum_{k=1}^K\phi_k} \\
P_{\theta}(x \mid y = k) &= \frac{1}{(2\pi)^{d/2}|\Sigma|^{d/2}}\exp\left(-\frac{1}{2}(x - \mu_k)^\top\Sigma_k^{-1}(x - \mu_k)\right)
\end{align*}$$


### **Optimizing the Log Likelihood**
Given a dataset $\mathcal{D}$, we want to optimize the log-likelihood $\ell(\theta)$:

$$\begin{align*}
\ell(\theta) &= \sum_{i=1}^n \log P_{\theta}(x^{(i)},y^{(i)}) = \sum_{i=1}^n \log P_{\theta}(x^{(i)} \mid y^{(i)}) + \sum_{i=1}^n\log P_{\theta}(y^{(i)}) \\
&= \sum_{k=1}^K \underbrace{\sum_{i:y^{(i)}=k}\log P(x^{(i)} \mid y^{(i)}; \mu_k, \Sigma_k)}_{\text{all the terms that involve} \; \mu_k, \Sigma_k} \; + \underbrace{\sum_{i=1}^n\log P(y^{(i)}; \vec{\phi})}_{\text{all the terms that involve} \; \vec{\phi}}
\end{align*}$$


Notice that each set of parameters $(\mu_k, \Sigma_k)$ is only found in one term of the summation over the $K$ classes and the $\phi_k$ is also in the same term.

Since each $(\mu_k, \Sigma_k)$ for $k= 1,2, \dots, K$ is only found in one term, optimization over $(\mu_k, \Sigma_k)$ can be carried out independently of all the other parameters by just looking at that term:
$$\begin{align*}
\max_{\mu_k, \Sigma_k}\sum_{i=1}^n \log P_{\theta}(x^{(i)}, y^{(i)}) &= \max_{\mu_k, \Sigma_k}\sum_{l=1}^K\sum_{i:y^{(i)}=l}\log P_{\theta}(x^{(i)} \mid y^{(i)}; \mu_l, \Sigma_l) \\
&= \max_{\mu_k, \Sigma_k}\sum_{i:y^{(i)}=k}\log P_{\theta}(x^{(i)} \mid y^{(i)}; \mu_k, \Sigma_k)
\end{align*}$$

Similarly, optimizing the $\vec{\phi} = (\phi_1, \phi_2, \dots, \phi_K)$ only involves a single term:
$$\max_{\vec{\phi}}\sum_{i=1}^n \log P_{\theta}(x^{(i)}, y^{(i)}; \theta) = \max_{\vec{\phi}}\sum_{i=1}^n\log P_{\theta}( y^{(i)};\vec{\phi})$$

### **Optimizing the Class Probabilities**
These observations greatly simplify the optimization of the model. Let's first consider the optimization over $\vec{\phi} = (\phi_1, \phi_2, \dots, \phi_K)$. From the previous analysis, our objective $J(\vec{\phi})$ equals:
$$\begin{align*}
J(\vec{\phi}) &= \max_{\vec{\phi}}\sum_{i=1}^n\log P_{\theta}( y^{(i)};\vec{\phi}) \\
&= \sum_{i=1}^n\log \phi_{y^{(i)}} - n\log\sum_{k=1}^K\phi_k \\
&= \sum_{k=1}^K \sum_{i:y^{(i)}=k}\log \phi_k - n \cdot \log\sum_{k=1}^K\phi_k
\end{align*}$$

Taking the derivative and setting it to zero, we obtain
$$\frac{\phi_k}{\sum_l\phi_l} = \frac{n_k}{n}$$

for each $k$, where $n_k = |\{i:y^{(i)}=k\}|$ is the number of training targets with class $k$.

Thus, the optimal $\phi_k$ is just the proportion of data points with class $k$ in the training dataset!

### **Optimizing the Conditional Probabilities**
Similarly, we can maximize the likelihood
$$\max_{\mu_k,\Sigma_k}\sum_{i:y^{(i)}=k}\log P(x^{(i)} \mid y^{(i)}; \mu_k, \Sigma_k) = \max_{\mu_k,\Sigma_k}\sum_{i:y^{(i)}=k}\log \mathcal{N}(x^{(i)} \mid \mu_k,\Sigma_k)$$

over the Guassian parameters.

Computing the derivative and setting it to zero, we obtain closed form solution:
$$\begin{align*}
\mu_k &= \frac{\sum_{i: y^{(i)} = k}x^{(i)}}{n_k} \\
\Sigma_k &= \frac{\sum_{i: y^{(i)} = k}(x^{(i)} - \mu_k)(x^{(i)} - \mu_k)^\top}{n_k}
\end{align*}$$

These are just empirical means and covariances of each class

### **Querying the Model**
How do we ask the model for predictions? As discussed earlier, we can apply Bayes' rule:
$$\arg\max_{\theta}P_{\theta}(y\mid x) = \arg\max_{\theta}P_{\theta}(x \mid y)P(y).$$

Thus, we can estimate the probability of $x$ and under each $P_{\theta}(x \mid y=k)P(y=k)$ and choose the class that explains the data best.

### **Classification Dataset: Iris Flowers**

To demonstrate this approach, we are going to use this Iris flower dataset.



In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import datasets

# Load the Iris dataset
iris = datasets.load_iris(as_frame=True)

# print part of the dataset
iris_X, iris_y = iris.data, iris.target
pd.concat([iris_X, iris_y], axis=1).head()

Visualize the dataset in 2D

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 4]

# create 2d version of dataset
X = iris_X.to_numpy()[:,:2]
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

# Plot also the training points
p1 = plt.scatter(X[:, 0], X[:, 1], c=iris_y, edgecolor='k', s=60, cmap=plt.cm.Paired)
plt.xlabel('Sepal Length (cm)')
plt.ylabel('Sepal Width (cm)')
plt.legend(handles=p1.legend_elements()[0], labels=['Setosa', 'Versicolour', 'Virginica'], loc='lower right')

### **Example: Iris Flowers Classification**

Let's see how this approach can be used in pratice on the Iris dataset

*   We will learn a good set of parameters for a Gaussian Disriminant Model
*   We will compare the outputs to the true predictions.



Let's first start by computing the true parameters on our dataset.

In [ ]:
# We can implement these formulas over the Iris dataset
d = 2 # number of features in our toy dataset
K = 3 #number of classes
n = X.shape[0] # size of dataset

# These are the shapes of the parameters
mus = np.zeros([K,d])
Sigmas = np.zeros([K,d,d])
phis = np.zeros([K])

# We now compute the parameters
for k in range(3):
  X_k = X[iris_y == k]
  mus[k] = np.mean(X_k, axis=0)
  Sigmas[k] = np.cov(X_k.T)
  phis[k] = X_k.shape[0]/float(n)

# print out the mean 
print(mus)


We can compute predictions using Bayes' rule

In [ ]:
# We can implement this in numpy
def gda_predictions(x,mus,Sigmas,phis):
  """ This returns class assignments and p(y|x) under the GDA model

  We compute \arg\max_y p(y|x) as \arg\max_y p(x|y)p(y)
  """
  # Adjust shapes
  n, d = x.shape
  x = np.reshape(x, (1,n,d,1))
  mus = np.reshape(mus, (K,1,d,1))
  Sigmas = np.reshape(Sigmas, (K,1,d,d))

  # Compute probabilities
  py = np.tile(phis.reshape((K,1)), (1,n)).reshape([K,n,1,1])
  pxy = (
      np.sqrt(np.abs((2*np.pi)**d*np.linalg.det(Sigmas))).reshape((K,1,1,1))
      * -.5*np.exp(
          np.matmul(np.matmul((x - mus).transpose([0,1,3,2]), np.linalg.inv(Sigmas)), x-mus)
      )
  )
  pyx = pxy * py
  return pyx.argmax(axis=0).flatten(), pyx.reshape([K,n])

idx, pyx = gda_predictions(X,mus,Sigmas,phis)

print(idx)


We can visualize the decision boundaries like we did earlier

In [ ]:
from matplotlib.colors import LogNorm
xx,yy = np.meshgrid(np.arange(x_min,x_max,.02), np.arange(y_min,y_max,.02))
Z, pyx = gda_predictions(np.c_[xx.ravel(), yy.ravel()], mus, Sigmas, phis)
logpy = np.log(-1./3*pyx)

# Put the result into color plot
Z = Z.reshape(xx.shape)
contours = np.zeros([K, xx.shape[0], xx.shape[1]])
for k in range(K):
  contours[k] = logpy[k].reshape(xx.shape)
plt.pcolormesh(xx,yy,Z,cmap=plt.cm.Paired)
for k in range(K):
  plt.contour(xx,yy,contours[k], levels=np.logspace(0,1,1))

# Plot also the training points
plt.scatter(X[:,0], X[:,1], c=iris_y, edgecolor='k', s=40, cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.show()

### **Algorithm: Gaussian Discriminant Analysis**


*   **Type**: Supervised Learning (multi-class classification)
*   **Model family**: Mixture of Gaussians
*   **Objective**: Log-likelihood
*   **Optimizer**: Closed form solution



### **Special cases of GDA**

Many important generative algorithms are special cases of GDA

*   **Linear Discriminant Ananlysis (LDA)**: all the covariance matrices $\Sigma_k$ take the same value.
*   **Gaussian Naive Bayes**: all the covariance matrices $\Sigma_k$ are diagonal.
*   **Quadratic Discriminant Analysis (QDA)**: another term for GDA





### **Generative vs. Discriminative Approaches**
Pros of discriminative models:
 * often more accurate because they make fewer modeling assumptions.

Pros of generative models:
 * Can do more than just prediction: generation, fill-in missing features, etc.
 * Can include extra prior knowledge; if prior knowledge is correct, model will be more accurate.
 * Often have closed form solutions, hence are faster to train.